**第7章**

In [1]:
from myABBR import *; import myUTIL as mu; from scipy.stats import norm

# 初期値 (pC:call=1 put=-1, futRT:先物金利, stkRT:行使金利)
tradeDT,            matDT,        pC, futPRC, stkPRC                 =\
jDT(2023,9,26), jDT(2023,12,15),  -1,  94.54,  94.50; setEvDT(tradeDT) 

# 50bp       5.4%         5.46%             5.5%      
volRT,      rfRT,        futRT,            stkRT                     =\
50/10000,  5.4/100, (100-futPRC)/100, (100-stkPRC)/100

# 予備計算: matYR:満期年, SD:Standard Deviation,  matDF:満期日discFactor
rfOBJ,_ = mu.ffTSH(tradeDT, rfRT, dcA360, cmpdSPL)
matYR   = dcA365.yearFraction(ql.Settings.instance().evaluationDate, matDT) 
SD      = volRT*np.sqrt(matYR)
matDF   = rfOBJ.discount(matDT)

# オプション価格
d1    = pC*(futRT-stkRT)/SD
optNPV= matDF*SD*(d1*norm.cdf(d1) + norm.pdf(d1))
# グリークス
optDL = pC*matDF*norm.cdf(d1)   ; optGM = matDF*norm.pdf(d1)/SD
optVG = matDF*np.sqrt(matYR)*norm.pdf(d1) 
optTH = rfRT*optNPV - 0.5*matDF*norm.pdf(d1)*volRT/np.sqrt(matYR)

print(f'optNPV:{optNPV:.6%} ', f'受渡金額:{optNPV*100*2500:.2f} \n', 
       f'optDL:{optDL:.2%} ' ,   f'optGM:{optGM :.2f} ', 
       f'optVG:{optVG:.2%} ' ,   f'optTH:{optTH:.2%}'              )

optNPV:0.113385%  受渡金額:283.46 
 optDL:-56.11%  optGM:165.97  optVG:18.19%  optTH:-0.20%


In [2]:
# 1.ソルバー用関数の準備 (価格差の算出)
def volSLVR(vol) :
    SD = vol*np.sqrt(matYR) ;   d1 = pC*(futRT-stkRT)/SD
    calcNPV= matDF*SD*(d1*norm.cdf(d1) + norm.pdf(d1))
    return tgtNPV-calcNPV
# 2.NPV 0.1133%のvol計算
tgtNPV = 0.1133/100             # accuracy, guess,  xMin,   xMax
impVOL = ql.Brent().solve(volSLVR, 1e-5,    0.001,  5e-5,   0.1)
                                # 0.1bp     10bp,   0.5bp,  10%
print(f'ImpVOL(tgtNPV={tgtNPV:.5%}): {impVOL:.4%}')

ImpVOL(tgtNPV=0.11330%): 0.4995%


In [3]:
#from myABBR import *     ; import myUTIL as mu
tradeDT = jDT(2023,9,26) ; setEvDT(tradeDT)
# 0.SOFRカーブ
crvDATA = [('depo','1d',5.31), ('swap','1m',5.32), ('swap','3m',5.38),
           ('swap','6m',5.46), ('swap','1y',5.45), ('swap','2y',5.01),
           ('swap','3y',4.67)]
sofrIX, sfCrvOBJ, sfCrvHDL, sfParRATE = mu.makeSofrCurve(crvDATA)

# 1.CME Term SOFR rate and Basis curve ( = all zero)
TsfRT3m  = 5.38558
TsfCrvBS = [('6m',0.0), ('1y',0.0), ('2y',0.0), ('3y',0.0) ]

# 2.TermSOFR指標金利オブジェクト
TsfCrvHDL= ql.RelinkableYieldTermStructureHandle()  
TsfIX    = ql.IborIndex('TermSofr', pdFreqQ, Tp2, usdFX, calUSs, mFLLW,
                                                          EoMt, dcA360, TsfCrvHDL)
# 3. Basis helperでのTermSOFRカーブオブジェクト
cHelper  = [ql.DepositRateHelper(mu.sqHDL(TsfRT3m/100),TsfIX)]
for tnr, bs in TsfCrvBS:
    cHelper.append(ql.OvernightIborBasisSwapRateHelper(mu.sqHDL(bs/100), 
                      pD(tnr), Tp2, calUSs, mFLLW, EoMf, sofrIX, TsfIX, sfCrvHDL))
TsfCrvOBJ = ql.PiecewiseLogLinearDiscount(Tp2, calUSs, cHelper, dcA360)
TsfCrvHDL.linkTo(TsfCrvOBJ) ; TsfCrvOBJ.enableExtrapolation()
# checking
print('決済日(reference):', TsfCrvOBJ.referenceDate().ISO())
[(dt.ISO(),df) for dt,df in TsfCrvOBJ.nodes()]

決済日(reference): 2023-09-28


[('2023-09-28', 1.0),
 ('2023-12-28', 0.9865692901876598),
 ('2024-03-28', 0.9731383278012513),
 ('2024-09-30', 0.947229008881084),
 ('2025-09-29', 0.9056140456298556),
 ('2026-09-28', 0.8708967171925004)]

In [4]:
# cap条件とSchedule
effDT,              matDT,      capSTK,   ntlAMT,     volRT          = \
jDT(2023,9,28), jDT(2024,9,28),  0.05,  10_000_000,  0.88/100

capSCD  = ql.Schedule(effDT,matDT,pdFreqQ,calUSs,mFLLW,mFLLW,dtGENb,EoMt)
print('Schedule: ', [dd.ISO() for dd in list(capSCD)]) # checking

# プライシング
capENG = ql.BachelierCapFloorEngine(sfCrvHDL, mu.sqHDL(volRT))   
capLeg = ql.IborLeg([ntlAMT], capSCD, TsfIX, dcA360)
capOBJ = ql.Cap(capLeg, [capSTK]) ; capOBJ.setPricingEngine(capENG)
print(f'capNPV  :   {capOBJ.NPV():,.2f}')

Schedule:  ['2023-09-28', '2023-12-28', '2024-03-28', '2024-06-28', '2024-09-30']
capNPV  :   42,659.65


In [5]:
# capOBJのcaplet用メソッド
dfLET = pd.DataFrame( dict(
            stdDEV = (capOBJ.optionletsStdDev()),        #stdDev=σ√T
            atmFWD = (capOBJ.optionletsAtmForward()),
            DF     = (capOBJ.optionletsDiscountFactor()), 
            NPV    = (capOBJ.optionletsPrice())           ))
dfLET.style.format(fmtSCF)

,stdDEV,atmFWD,DF,NPV
0,0.000000,5.385580%,0.98629210,"9,613.00"
1,0.004394,5.460013%,0.97285119,"12,135.07"
2,0.006214,5.257983%,0.95995225,"9,762.74"
3,0.007625,5.258748%,0.94694952,"11,148.83"


In [6]:
# caplet日付関連
dfCap = pd.DataFrame({
    'matYR':      dcA365.yearFraction(tradeDT, cpn.fixingDate()), #maturity year
    'fixingDate': cpn.fixingDate().ISO(),
    'accruStart': cpn.accrualStartDate().ISO(),
    'accruEnd':   cpn.accrualEndDate().ISO(),
    'payDate':    cpn.date().ISO(),
    'days':       dcA360.dayCount(cpn.accrualStartDate(),cpn.accrualEndDate()),
    'TsfDF':      TsfCrvOBJ.discount(cpn.date())            #calc forward rate 
    } for cpn in map(ql.as_floating_rate_coupon, capLeg)) 

dfCap = pd.concat([dfCap, dfLET], axis=1)
fmtSCF.update(TsfDF='{:.8f}') ; dfCap.style.format(fmtSCF)

,matYR,fixingDate,accruStart,accruEnd,payDate,days,TsfDF,stdDEV,atmFWD,DF,NPV
0,0.0000,2023-09-26,2023-09-28,2023-12-28,2023-12-28,91,0.98656929,0.000000,5.385580%,0.98629210,"9,613.00"
1,0.2493,2023-12-26,2023-12-28,2024-03-28,2024-03-28,91,0.97313833,0.004394,5.460013%,0.97285119,"12,135.07"
2,0.4986,2024-03-26,2024-03-28,2024-06-28,2024-06-28,92,0.96023558,0.006214,5.257983%,0.95995225,"9,762.74"
3,0.7507,2024-06-26,2024-06-28,2024-09-30,2024-09-30,94,0.94722901,0.007625,5.258748%,0.94694952,"11,148.83"


In [7]:
# 原資産スワップ スケジュール
swEffDT, swMatDT = jDT(2024,9,30), jDT(2026,9,30) 
fixSCD = ql.Schedule(swEffDT,swMatDT,pdFreqA,calUSs,mFLLW,mFLLW,dtGENb,EoMf)
# for checking
scdDTs = [dd.ISO() for dd in fixSCD]              ; print('fixSCD:',scdDTs)
DFs    = [sfCrvOBJ.discount(dd) for dd in fixSCD] ; print('   DFs:',nA(DFs))

fixSCD: ['2024-09-30', '2025-09-30', '2026-09-30']
   DFs: [0.94695 0.90525 0.87045]


In [8]:
# アニュイティ、フォワードスワップレート
Annu = mu.calcAnnuity(fixSCD, sfCrvOBJ, dcA360)
fwdRT = (DFs[0]-DFs[-1])/Annu  
print(f'アニュイティ:{Annu:.5f}, フォワードスワップレート:{fwdRT:.5%}')

アニュイティ:1.80036, フォワードスワップレート:4.24896%


In [9]:
#from myABBR import * ; import myUTIL as mu
# 0.SOFRカーブ
crvDATA = [('depo','1d',5.31), ('swap','1m',5.32), ('swap','3m',5.38),
           ('swap','6m',5.46), ('swap','1y',5.45), ('swap','2y',5.01),
           ('swap','3y',4.67)]
sofrIX, sfCrvOBJ, sfCrvHDL, sfParRATE = mu.makeSofrCurve(crvDATA)

# 1. 原資産条件、オプション条件、ボラティリティ等
tradeDT,           exprDT,         swEffDT,         swMatDT,     pC      =\
jDT(2023,9,26), jDT(2024,9,26), jDT(2024,9,30), jDT(2026,9,30),   1

fwdRT,      spdRT, payLag,         volHDL,       ntlAMT                  =\
4.24896/100, 0.0,    0,   mu.sqHDL(1.35/100), 10_000_000;  setEvDT(tradeDT)

# 2.原資産オブジェクト
fixSCD = ql.Schedule(swEffDT,swMatDT,pdFreqA,calUSs,mFLLW,mFLLW,dtGENb,EoMf)
swapOBJ= ql.OvernightIndexedSwap(
                   pC, ntlAMT, fixSCD, fwdRT, dcA360, sofrIX, spdRT, payLag)
# 3. swaption計算
swptnOBJ = ql.Swaption(swapOBJ, ql.EuropeanExercise(exprDT))
swptnENG = ql.BachelierSwaptionEngine(sfCrvHDL, volHDL)
swptnOBJ.setPricingEngine(swptnENG)
print(         f'NPV:{swptnOBJ.NPV()           :,.2f}\n'
        f'delta(1bp):{swptnOBJ.delta()  /ntlAMT:,.2%}, '
         f'vega(1bp):{swptnOBJ.vega()   /ntlAMT:,.2%}, '
           f'annuity:{swptnOBJ.annuity()/ntlAMT:,.5f}'  ) 

NPV:97,095.16
delta(1bp):90.02%, vega(1bp):71.92%, annuity:1.80036


In [10]:
# クロージャー(関数内関数)によるノーマルモデル
def normalCALC(payOffOBJ, matDT, futRT, volRT, rfOBJ):
    # 満期年
    trdDT_ = ql.Settings.instance().evaluationDate        
    matYR_ = dcA365.yearFraction(trdDT_, matDT)
    # ペイオフ
    pC_    = payOffOBJ.optionType()
    stkRT_ = payOffOBJ.strike()
    # 満期日DF、カーブオブジェクト
    matDF_ = rfOBJ.discount(matDT)
    rfOBJ_ = rfOBJ
    # ボラ、原資産価格
    volRT_ = volRT
    futRT_ = futRT

    # SD: 標準偏差関数
    def SD(volRT=volRT_, matYR=matYR_): return volRT*np.sqrt(matYR)
    # dd: d1関数
    def dd(futRT=futRT_, volRT=volRT_, matYR = matYR_): 
        return pC_*(futRT- stkRT_)/SD(volRT,matYR)
    # npvの計算  (クロージャー)
    def npv(futRT=futRT_, volRT=volRT_, matYR = matYR_):
        matDF = matDF_ if matYR == matYR_ else \
                          rfOBJ_.discount(dcA360.yearFraction(trdDT_, matDT))
        d1 = dd(futRT, volRT, matYR) 
        return matDF*SD(volRT,matYR)*(d1*norm.cdf(d1) + norm.pdf(d1))
    
    return npv # 関数オブジェクトを戻す

In [11]:
#from myABBR import * ; import myUTIL as mu; from scipy.stats import norm

# 初期値 (pC:call=1 put=-1, futRT:先物金利, stkRT:行使金利)
tradeDT,            matDT,       pC, futPRC, stkPRC                     =\
jDT(2023,9,26), jDT(2023,12,15), -1,  94.54,  94.50 ; setEvDT(tradeDT)

volRT,      rfRT,        futRT,            stkRT                         =\
50/10000,  5.4/100, (100-futPRC)/100, (100-stkPRC)/100

# 引数用オブジェクト準備
rfOBJ,_ = mu.ffTSH(tradeDT, rfRT, dcA360, cmpdSPL)
payOFF  = ql.PlainVanillaPayoff(pC,stkRT)
# 計算
npvFC = normalCALC(payOFF, matDT, futRT, volRT, rfOBJ)
print(f'npv:{npvFC():.5%} at 初期値')

npv:0.11338% at 初期値


In [12]:
# 先物金利1%up
futRTup = futRT*1.01
print(f'npv:{npvFC(futRTup):.5%} at 金利{futRTup:.3%}', end = ',  ')
# 先物ボラ1%up
volRTup = volRT+0.01
print(f'npv:{npvFC(volRT=volRTup):.5%} at vol {volRTup:.3%}', end = ',  ')
# 1日経過
matYR1d = dcA365.yearFraction(tradeDT+1, matDT)
print(f'npv:{npvFC(matYR=matYR1d):.5%} at {(tradeDT+1).ISO()}')

npv:0.08524% at 金利5.515%,  npv:0.29705% at vol 1.500%,  npv:0.11281% at 2023-09-27


In [13]:
class normalCalculator:
    def __init__(self, payOffOBJ, matDT, futRT, volRT, rfOBJ):
      # 満期年
      self.trdDT_ = ql.Settings.instance().evaluationDate        
      self.matYR_ =   dcA365.yearFraction(self.trdDT_, matDT)
      # ペイオフ
      self.pC_    = payOffOBJ.optionType()
      self.stkRT_ = payOffOBJ.strike()
      # 満期日DF、カーブオブジェクト
      self.matDF_ = rfOBJ.discount(matDT)
      self.rfOBJ_ = rfOBJ
      # ボラ、原資産価格
      self.volRT_ = volRT
      self.futRT_ = futRT

    # SD: 標準偏差メソッド
    def SD(self, volRT=None, matYR=None): 
        volRT = volRT or self.volRT_ ; matYR = matYR or self.matYR_
        return volRT*np.sqrt(matYR)
    # dd: d1メソッド
    def dd(self, futRT=None, volRT=None, matYR=None): 
        futRT = futRT or self.futRT_ ; volRT = volRT or self.volRT_
        matYR = matYR or self.matYR_
        return self.pC_*(futRT - self.stkRT_)/self.SD(volRT, matYR)
    # npvメソッド
    def npv(self, futRT=None, volRT=None, matYR=None):
        futRT = futRT or self.futRT_; volRT = volRT or self.volRT_
        matYR = matYR or self.matYR_
        matDF = self.matDF_ if matYR == self.matYR_ else\
                 self.rfOBJ_.discount(dcA360.yearFraction(self.trdDT_, matDT))
        d1 = self.dd(futRT, volRT, matYR) 
        return matDF*self.SD(volRT,matYR)*(d1*norm.cdf(d1) + norm.pdf(d1))

In [14]:
#from myABBR import * ; import myUtil as mu ; from scipy.stats import norm

# 初期値 (pC:call=1 put=-1, futRT:先物金利, stkRT:行使金利)
tradeDT,            matDT,       pC, futPRC, stkPRC                    =\
jDT(2023,9,26), jDT(2023,12,15), -1,  94.54,  94.50 ; setEvDT(tradeDT)

volRT,      rfRT,        futRT,            stkRT                        =\
50/10000,  5.4/100, (100-futPRC)/100, (100-stkPRC)/100

rfOBJ,rfHDL = mu.ffTSH(tradeDT, rfRT, dcA360, cmpdSPL)
payOFF = ql.PlainVanillaPayoff(pC,stkRT)

# コンストラクタ
optOBJ = normalCalculator(payOFF, matDT, futRT, volRT, rfOBJ)
print(  f'npv:{optOBJ.npv():.5%} at 初期値')

npv:0.11338% at 初期値


In [15]:
# 先物金利1%up
futRTup = futRT*1.01
print(f'npv:{optOBJ.npv(futRTup):.5%} at 金利{futRTup:.3%}', end = ',  ')
# 先物ボラ1%up
volRTup = volRT+0.01
print(f'npv:{optOBJ.npv(volRT=volRTup):.5%} at vol {volRTup:.3%}', end = ',  ')
# 1日経過
matYR1d = dcA365.yearFraction(tradeDT+1, matDT)
print(f'npv:{optOBJ.npv(matYR=matYR1d):.5%} at {(tradeDT+1).ISO()}')

npv:0.08524% at 金利5.515%,  npv:0.29705% at vol 1.500%,  npv:0.11281% at 2023-09-27


In [16]:
x=100  ; y=200 ; print(x or y, end=" , ")
x=None ; y=200 ; print(x or y)

100 , 200


In [17]:
from math import sqrt, log; from scipy.stats import norm

def PhiTilde(x): return norm.cdf(x) + norm.pdf(x) / x

def inversePhiTilde(PhiTildeStar):
    if (PhiTildeStar < -0.001882039271):
        g = 1.0 / (PhiTildeStar - 0.5)
        xibar = (0.032114372355 - g*g*(0.016969777977 - \
                                        g*g*(2.6207332461E-3 - 9.6066952861E-5*g*g))) \
                / (1.0 - g*g*(0.6635646938 - g*g*(0.14528712196 - 0.010472855461*g*g)))
        xbar = g*(0.3989422804014326 + xibar*g*g)
    else:
        h = sqrt(-log(-PhiTildeStar))
        xbar = (9.4883409779 - h*(9.6320903635 - h*(0.58556997323 + 2.1464093351*h))) \
                 / (1.0 - h*(0.65174820867 + h*(1.5120247828 + 6.6437847132E-5*h)))
    
    q = (PhiTilde(xbar) - PhiTildeStar) / norm.pdf(xbar)
    xstar = xbar + 3.0*q*xbar*xbar*(2.0 - q*xbar*(2.0 + xbar*xbar)) \
        /(6.0 + q*xbar*(-12.0 + xbar*(6.0*q + xbar*(-6.0 + q*xbar*(3.0 + xbar*xbar)))))
    return xstar

def normalVol(pC, stkRT, futRT, matYR, optNPV, DF):
    # DF=1に修正
    optNPV = optNPV/DF
    # ATM
    if (abs(stkRT - futRT) < 1E-15):
        return optNPV / (sqrt(matYR) * norm.pdf(0.0))
    # ATM以外
    timeValue    = optNPV - max(pC*(futRT - stkRT), 0.0)
    if (abs(timeValue) < 1E-15): return 0.0
    
    PhiTildeStar = -abs(timeValue / (stkRT-futRT))
    xstar        = inversePhiTilde(PhiTildeStar)
    return abs((stkRT-futRT) / (xstar*sqrt(matYR)))
        
normalVol(-1, 0.055, 0.0546, 0.21918, 0.0011330, 0.98814)
        # pC, stkRT,  futRT,  matYR,    optNPV,     DF

np.float64(0.004995328817556409)